<a href="https://colab.research.google.com/github/teang1995/satellite_image_classification/blob/master/ResNet_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**import Libraries**

In [0]:
import os
import shutil
from PIL import Image
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
from torchvision import transforms as T
import torchvision
import cv2
import sys
from google.colab import drive
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
import pandas as pd
from torch.utils.tensorboard import SummaryWriter

**check cuda**

In [0]:
torch.cuda.is_available()

True

**drive mount**

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
writer = SummaryWriter("runs/simple_classification")

In [0]:
classes = ['thermal_power_station', 'stadium', 'tennis_court', 'terrace', 'storage_tank', 'sea_ice', 'wetland', 'sparse_residential', 'ship', 'snowberg', 'rectangular_farmland', 'runway', 'parking_lot', 'overpass', 'roundabout', 'palace', 'railway_station', 'railway', 'mountain', 'river', 'ground_track_field', 'medium_residential', 'mobile_home_park', 'island', 'golf_course', 'intersection', 'lake', 'harbor', 'industrial_area', 'meadow', 'forest', 'bridge', 'church', 'freeway', 'dense_residential', 'chaparral', 'desert', 'commercial_area', 'circular_farmland', 'cloud', 'airport', 'airplane', 'beach', 'baseball_diamond', 'basketball_court']

**Dataset Class**

In [0]:
class NWPU_RESISC45(Dataset):
  '''
  1. Satellite image dataset
  2. 45 classes
  '''
  #initiallize my data , download
  def __init__(self, root, phase = 'Train',transform = None):
    self.phase = phase
    imgs = pd.read_csv(root) # csv파일의 한 행
    os.chdir("/content/drive/My Drive/dataset" + "/" + self.phase)
    self.img_list = [os.path.join(imgs["img_name"][i]) for i in range(len(imgs))] #파일 경로로 리스트를 만듦.
    self.label_list = [os.path.join(imgs["label"][i]) for i in range(len(imgs))] #라벨 리스트를 만듦
    #self.imgs = np.random.permutation(imgs) #dataloader의 shuffle = True로 대체
    self.transform = transform
  #csv파일에서 따오기로 결정
  def __getitem__(self,index):
    global classes
    os.chdir("/content/drive/My Drive/dataset" + "/" + self.phase)
    sample = self.img_list[index]
    img_path = sample
    data = Image.open(img_path)
    #data = data.convert('L') #binary chaneel
    data = self.transform(data)
    label = classes.index(self.label_list[index])
    return data.float(), label

  def __len__(self):
    return len(self.img_list)

**transform and Dataloader**

In [0]:
transform = T.Compose([
                       T.Resize(224),
                       T.RandomHorizontalFlip(),                    #data
                       #T.RandomCrop(size = [224,224],padding = 4),  #augmentation
                       T.ToTensor()])
batch_size = 32
valid_batch_size = 8
test_batch_size = 14
trainset = NWPU_RESISC45(root = "/content/drive/My Drive/dataset/Train.csv",phase = 'Train',transform = transform)
trainloader = DataLoader(trainset,batch_size = batch_size, shuffle = True)

validationset = NWPU_RESISC45(root = "/content/drive/My Drive/dataset/Validation.csv",phase = 'Validation',transform = transform)
validloader = DataLoader(validationset, batch_size = valid_batch_size, shuffle = True)

testset = NWPU_RESISC45(root = "/content/drive/My Drive/dataset/Test.csv",phase = 'Test',transform = transform)
testloader = DataLoader(testset,batch_size = test_batch_size, shuffle = True)

data_loader = {"train" : trainloader , "validation":validloader}

**VGGNET**

In [0]:
class Net(nn.Module) :
  def __init__(self):
    super(Net,self).__init__()
    self.layer1 = nn.Sequential(nn.Conv2d(3,64,kernel_size = 7,stride = 2,padding = 3,bias = True),
                                nn.BatchNorm2d(64),nn.ReLU(inplace = True),
                                nn.MaxPool2d(kernel_size = 3,stride = 2, padding = 1))
    
    self.layer2 = nn.Sequential(nn.Conv2d(64,64,kernel_size = 3,stride = 1,padding = 1,bias = True),
                                nn.BatchNorm2d(64),nn.ReLU(),
                                nn.Conv2d(64,64,kernel_size = 3, stride = 1, padding = 1, bias = True),
                                nn.BatchNorm2d(64),nn.ReLU())
    
    self.layer3 = nn.Sequential(nn.Conv2d(64,128, kernel_size = 3, stride = 2, padding = 1, bias = True),
                                nn.BatchNorm2d(128),nn.ReLU(),
                                nn.Conv2d(128,128, kernel_size = 3, stride = 1, padding = 1, bias = True),
                                nn.BatchNorm2d(128),nn.ReLU())
  
    self.layer4= nn.Sequential(nn.ReLU(),
                                nn.Conv2d(128,128,kernel_size = 3, stride = 1, padding = 1, bias = True),
                                nn.BatchNorm2d(128),nn.ReLU(),
                                nn.Conv2d(128,128, kernel_size = 3, stride = 1, padding = 1, bias = True),
                                nn.BatchNorm2d(128),nn.ReLU())
    

    self.layer5 = nn.Sequential(nn.Conv2d(128,256, kernel_size = 3, stride = 2, padding = 1, bias = True),
                                nn.BatchNorm2d(256),nn.ReLU(),
                                nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1, bias = True),
                                nn.BatchNorm2d(256),nn.ReLU())
    self.layer6 = nn.Sequential(nn.ReLU(),
                                nn.Conv2d(256,256,kernel_size = 3, stride = 1, padding = 1, bias = True),
                                nn.BatchNorm2d(256),nn.ReLU(),
                                nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1, bias = True),
                                nn.BatchNorm2d(256),nn.ReLU())
    self.layer7 = nn.Sequential(nn.Conv2d(256,512, kernel_size = 3, stride = 2, padding = 1, bias = True),
                                nn.BatchNorm2d(512),nn.ReLU(),
                                nn.Conv2d(512,512, kernel_size = 3, stride = 1, padding = 1, bias = True),
                                nn.BatchNorm2d(512),nn.ReLU())
    
    self.layer8 = nn.Sequential(nn.ReLU(),
                                nn.Conv2d(512,512, kernel_size = 3, stride = 1, padding = 1, bias = True),
                                nn.BatchNorm2d(512),nn.ReLU(),
                                nn.Conv2d(512,512, kernel_size = 3, stride = 1, padding = 1, bias = True),
                                nn.BatchNorm2d(512),nn.ReLU())
    self.gap = nn.AdaptiveAvgPool2d((1,1))

    self.linear = nn.Sequential(
        nn.Linear(512, 1000),
        nn.Linear(1000,45))
    
    #Kaiming He initialization
    for m in self.modules():
      if type(m) == nn.Linear or type(m) == nn.Conv2d :
        init.kaiming_normal_(m.weight.data)
        init.constant_(m.bias.data,0)

  def forward(self,x):
    
    x = self.layer1(x)
    y = self.layer2(x)
    x = x + y
    y = self.layer2(x)
    x = x + y
    x = self.layer3(x)
    y = self.layer4(x)
    x = x + y
    x = self.layer5(x)
    y = self.layer6(x)
    x = x + y
    x = self.layer7(x)
    y = self.layer8(x)
    x = x + y
    #print(len(x), len(x[0]),len(x[0][0]),len(x[0][0][0]))
    x = self.gap(x)
    #print(len(x), len(x[0]),len(x[0][0]),len(x[0][0][0]))
    x = x.view(x.size(0),-1)
    x = self.linear(x)
    return F.log_softmax(x)
#cuda사용 가능하면 gpu사용, 아니면 cpu로 연산
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = Net()
net = net.cuda()

**optimizer and loss function**

In [0]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(),lr = 0.1,momentum = 0.9, weight_decay = 0.0001)

**Train and Validation**

In [0]:
before = 999999.0
learning_rate = 0.1
PATH = "/content/drive/My Drive/weight.txt"
torch.save(net.state_dict(), PATH)
net.load_state_dict(torch.load(PATH))
print("빰")
for epoch in range(50):
  print('Epoch {}/{}'.format(epoch + 1,50))
  print('-' * 10)
  for phase in ["train" , "validation" ]:
    #running_loss는 학습 현황을 보이기 위한 loss
    #total_loss는 한 epoch단위로 valid_loss가 증가하면 learning rate를 감소시키기 위함.
    running_loss = 0.0
    total_loss = 0.0
    if phase == "train":
      net.train(True)
    else:
      net.train(False)    
    for i,data in enumerate(data_loader[phase]):
      inputs,labels = data
      inputs = inputs.cuda()
      labels = labels.cuda()
      optimizer.zero_grad()
      outputs = net(inputs)
      #아래의 내용은 train accuracy를 위함
      loss = criterion(outputs,labels)
      if phase == "train":
        loss.backward()
        optimizer.step()

      running_loss += loss.item() 
      if phase == "validation":         #validation 단계에서
        total_loss += loss.item()       #total_loss 갱신
      if i % 10 == 9:   
        print("[%d, %5d] %s loss : %.3f" %(epoch + 1, i + 1, phase , running_loss / 10))
        writer.add_scalar(phase + "loss", running_loss/10, epoch * len(data_loader[phase]) + i)
        running_loss = 0.0
  
  #save model weights and bias
  torch.save(model.state_dict(), PATH)
  #test
  correct = 0
  for data in testloader:
    inputs,labels = data
    inputs,labels = Variable(inputs.cuda()),Variable(labels.cuda())
    outputs = net(inputs)
    pred = outputs.data.max(1,keepdim = True)[1]
    correct += pred.eq(labels.data.view_as(pred)).sum()
  print('Accuracy: {}/3150 ({:.0f}%)\n'.format(correct,100. * correct / 3150))

  #validation loss가 커지면
  if True :
    total_loss = 999999.0
    #optimizer의 learning rate를 10으로 나눈다.
    learning_rate = learning_rate / 10
    optimizer = optim.SGD(net.parameters(),lr = lr,momentum = 0.9, weight_decay = 0.0001)
    #weight값을 이전의 것으로 갱신한다.
    net.load_state_dict(torch.load(PATH))